In [1]:
from julia.api import Julia
jl = Julia(compiled_modules=False)
jl.eval("using OnlineStats,MAT")
from julia import Main

In [2]:
Main.eval("""
#Function to generate number of bins (Logarithmically spaced) Given an array length
# I.E. from 2 bins (log(10,2.5)) for the entire length to As many bins that can be fitted in that size of the array
#The case of 1 bin spanning the entire array and Same number of bin as entire length of the array is Manually Added


#https://stackoverflow.com/questions/12418234/logarithmically-spaced-integers
#The values in the array generated by logspace(0, k, n) are the numbers 10^{ik / (n−1)} for 0 ≤ i < n

#Generating Logarithmic spaced numbers 
log_space(st,sp,n,Integ=false,Base=10) = (Integ==true ? (unique(Int64.(floor.(Base .^(range(st, stop=sp, length=n)))))) : (Base .^(range(st, stop=sp, length=n))))

#Generating Logarithmic spaced Integears between  
#log(10,2.5) ----------> 2 Bins
#log(10,endofLine) ----> number of bins nearest to the array-length provided
function GenBinNums(endofLine,num_of_bins=100)
    LogSpacedArray=log_space(log(10,2.5),log(10,endofLine),num_of_bins,true)
    #("Asked for: ",num_of_bins," Digits! Returning: ",length(LogSpacedArray)," Digits!")
    return LogSpacedArray 
end
#GenBinNums(500,10)  #For array of length 500, We asked of 10 bin counts which are logarithmcally spaced

function resrel(signal::Array,time_bins::Int)
    num_of_datapoints=size(signal)[1]
    
    #Creating Balanced Bins
    #See: http://stackoverflow.com/a/21767522
    bins=min(num_of_datapoints,time_bins);
    floor =num_of_datapoints ÷ bins;
    ceiling = floor + 1;
    stepdown = num_of_datapoints % bins;
    lm=ceiling*stepdown;
    
    #Counting Spikes
    ###################################################################
     if(num_of_datapoints % bins==false)
         counts=[sum(elements) for elements in Iterators.partition(signal[:], num_of_datapoints ÷ bins)]
     else
        c1=[sum(elements) for elements in Iterators.partition(signal[1:lm], ceiling)]
        c2=[sum(elements) for elements in Iterators.partition(signal[lm+1:end], floor)]
        counts=vcat(c1,c2)
   ###################################################################     
    end
    #println(sum(counts))
    #return counts
    
    #Eleminating Zero-counts
    zero_loc=findall(counts.==0); #Finding Zero Locations
    deleteat!(counts,zero_loc); #Eleminating Zeros
    
    #Counting unique values and Frequencies
    o=fit!(CountMap(Int), counts); # Couting Unique appearences
    vals=collect(keys(o));freq=collect(values(o)) # Values & their Freqiencies
    
    #for elements in zip(vals,freq); println(elements); end # See
    
    #Checking if total count and Freq/Values matches or not
    sum(counts) == sum(vals.*freq) || throw(AssertionError("Value/Frequency coumputation error!"))
    
    #Calculating Hs & Hk
    M=sum(vals.*freq)
    hs= - sum(  ((freq.*vals)./M) .* log.(M,(vals./M)) )
    hk= - sum(  ((freq.*vals)./M) .* log.(M,((freq.*vals)./M)) )
    
    return [hs,hk]
end

function doit(signal,nbbins)
    result=[resrel(signal,bins) for bins in nbbins]
    return vcat(result'...)
end  
#Sorting and adding theoretical Extremas
function prep(HofS,HofK)
        SHofS=HofS[sortperm(HofS)]
        SHofK=HofK[sortperm(HofS)]

        #Adding Theoretical minimum
        insert!(SHofS,1,0.0);
        insert!(SHofK,1,0.0);

        #Adding Theoretical maximum
        append!(SHofS,1.0);
        append!(SHofK,0.0);
    return SHofS,SHofK
end
function calculate_area(data_points)
    # calculates integral using the trapezoid rule
    return sum(0.5*(abs.(data_points[:,1][1:end-1]+data_points[:,1][2:end]))
        .*(abs.(data_points[:,2][1:end-1]-data_points[:,2][2:end])))
end

function calculate_MSR(binned_spikes)
    nbinsM=GenBinNums(size(binned_spikes)[1],100);
    HSHKm=doit(binned_spikes,nbinsM);
    MySHofS,MySHofK=prep(HSHKm[:,1],HSHKm[:,2]);
    myMSR=calculate_area(hcat(MySHofK,MySHofS));
    return myMSR
end
""")

<PyCall.jlwrap calculate_MSR>